In [4]:
import dask.dataframe as dd

df = dd.read_csv('../../data/flight_data/flight_data.csv', sample_rows=1000)
df.head(10)

ValueError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: EOF encountered while reading header. 
Pass argument `sample_rows` and make sure the value of `sample` is large enough to accommodate that many rows of data

In [5]:
import pandas as pd

df = pd.read_csv('../../data/flight_data/flight_data.csv')
df.head(10)

,icao24,callsign,estdepartureairport,estarrivalairport,firstseen,lastseen,flight_duration,track
0,8963df,UAE135,OMDB,LIPZ,1735711201,1735732155,20954,"[array([ 1.73573215e+09, 4.54877278e+01, 1.2..."
1,aaa326,AAL124,NaN,KDFW,1735711202,1735735435,24233,"[array([ 1.73573543e+09, 3.28514953e+01, -9.7..."
2,868b50,ANA380,NaN,NaN,1735711203,1735716313,5110,"[array([1.73571631e+09, 3.51708674e+01, 1.3507..."
3,78080d,CQH6194,VTBD,NaN,1735711204,1735719171,7967,"[array([1.73571917e+09, 2.29406635e+01, 1.1444..."
4,78052f,CXA833,ZSAM,VTBS,1735711204,1735723919,12715,"[array([1.73572392e+09, 1.36619110e+01, 1.0072..."
5,800b30,AIC214,NaN,VIDP,1735711204,1735713142,1938,"[array([1.73571314e+09, 2.85390822e+01, 7.7103..."
6,4ca2d5,RYR7WA,LEAL,EHEH,1735711206,1735719539,8333,"[array([1.73571953e+09, 5.14545135e+01, 5.3794..."
7,780aa7,CRK237,NaN,VHHH,1735711212,1735716535,5323,"[array([ 1.73571653e+09, 2.23236237e+01, 1.1..."
8,4cadbe,RYR2NQ,EGGD,LHBP,1735711213,1735719449,8236,"[array([ 1.73571944e+09, 4.74261932e+01, 1.9..."
9,7c46e8,OAI,YBMC,YBMC,1735711214,1735712303,1089,"[array([ 1.73571230e+09, -2.65682831e+01, 1.5..."


(time: 1735732150, latitude: 45.487727795617054, longitude: 12.329540717892531, altitude: -304.8, heading: 42.04922, onground: False)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36761 entries, 0 to 36760
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   icao24               36761 non-null  object
 1   callsign             36730 non-null  object
 2   estdepartureairport  27386 non-null  object
 3   estarrivalairport    29657 non-null  object
 4   firstseen            36761 non-null  int64 
 5   lastseen             36761 non-null  int64 
 6   flight_duration      36761 non-null  int64 
 7   track                36761 non-null  object
dtypes: int64(3), object(5)
memory usage: 2.2+ MB


In [9]:
df.iloc[0]['track']

'[array([ 1.73573215e+09,  4.54877278e+01,  1.23295407e+01, -3.04800000e+02,\n         4.20492200e+01,  0.00000000e+00])\n array([ 1.73573213e+09,  4.54794880e+01,  1.23189563e+01, -3.04800000e+02,\n         4.17379650e+01,  0.00000000e+00])\n array([1.73573205e+09, 4.54430376e+01, 1.22721974e+01, 0.00000000e+00,\n        4.21087300e+01, 0.00000000e+00])\n array([1.73573199e+09, 4.54064941e+01, 1.22256252e+01, 3.04800000e+02,\n        4.14236640e+01, 0.00000000e+00])\n array([1.73573198e+09, 4.54026303e+01, 1.22208163e+01, 6.09600000e+02,\n        3.96022530e+01, 0.00000000e+00])\n array([1.73573198e+09, 4.54007263e+01, 1.22185625e+01, 6.09600000e+02,\n        3.75685920e+01, 0.00000000e+00])\n array([1.73573198e+09, 4.53995819e+01, 1.22173200e+01, 6.09600000e+02,\n        3.55376780e+01, 0.00000000e+00])\n array([1.73573197e+09, 4.53981628e+01, 1.22158160e+01, 6.09600000e+02,\n        3.32500150e+01, 0.00000000e+00])\n array([1.73573197e+09, 4.53973846e+01, 1.22150312e+01, 6.09600000e

In [11]:
import numpy as np
import pandas as pd
import ast
import re

def parse_track(track_str):
    """
    Given a track string where each state vector is represented as:
      array([number1, number2, number3, number4, number5, number6])
    and multiple state vectors are concatenated (possibly separated by newlines),
    this function returns a list of tuples where each tuple represents a state vector.

    Example input:
      '[array([ 1.73573215e+09,  4.54877278e+01,  1.23295407e+01, -3.04800000e+02,
               4.20492200e+01,  0.00000000e+00])
        array([ 1.73573213e+09,  4.54794880e+01,  1.23189563e+01, -3.04800000e+02,
               4.17379650e+01,  0.00000000e+00])
        ... ]'
    """
    # Using regex to find all patterns of the form array([...])
    matches = re.findall(r"array\(\[([^\]]+)\]\)", track_str)
    result = []
    for match in matches:
        # Remove any newlines and extra spaces, and then split on comma.
        # This yields a list of number strings.
        numbers_str_list = match.replace("\n", " ").split(',')
        try:
            # Convert each number string to a float and form a tuple.
            numbers_tuple = tuple(float(num.strip()) for num in numbers_str_list)
            result.append(numbers_tuple)
        except ValueError as e:
            print("Error converting numbers:", e)
            continue
    return result

def compute_track_metrics(track):
    """
    Given a track as a list/array of state vectors,
    compute aggregate metrics such as min, max, average for numeric fields,
    and convert the minimum and maximum times (start and end) into human readable datetime.
    
    Each state vector is assumed to be in the following order:
      [time, latitude, longitude, altitude, heading, onground]
    
    Returns a dictionary with the aggregate metrics.
    """
    # Ensure the track is not None and has data
    if not track or len(track) == 0:
        return None
    
    # If needed, convert track into a numpy array.
    # We use object dtype in case some values (like onground) are booleans.
    arr = np.array(track, dtype=object)
    
    # Separate out the values. We assume that time, latitude, longitude, altitude, heading are numeric.
    # onground is assumed to be boolean (or convertible to int).
    times = np.array([float(item) for item in arr[:, 0]])
    latitudes = np.array([float(item) for item in arr[:, 1]])
    longitudes = np.array([float(item) for item in arr[:, 2]])
    altitudes = np.array([float(item) for item in arr[:, 3]])
    headings = np.array([float(item) for item in arr[:, 4]])
    
    # For onground, we convert to int(0,1) to calculate a fraction True if needed.
    try:
        onground = np.array([int(item) for item in arr[:, 5]])
        onground_frac = onground.mean() * 100  # percentage of time that the flight was on ground
    except Exception:
        onground_frac = None
    
    metrics = {
        "time_min": times.min(),
        "time_max": times.max(),
        "time_avg": times.mean(),
        "latitude_min": latitudes.min(),
        "latitude_max": latitudes.max(),
        "latitude_avg": latitudes.mean(),
        "longitude_min": longitudes.min(),
        "longitude_max": longitudes.max(),
        "longitude_avg": longitudes.mean(),
        "altitude_min": altitudes.min(),
        "altitude_max": altitudes.max(),
        "altitude_avg": altitudes.mean(),
        "heading_min": headings.min(),
        "heading_max": headings.max(),
        "heading_avg": headings.mean(),
        "onground_percentage": onground_frac,
        # Convert Unix time to human readable datetime
        "start_time": pd.to_datetime(times.min(), unit="s"),
        "end_time": pd.to_datetime(times.max(), unit="s")
    }
    
    return metrics


In [12]:

# Example of applying this function to a single row from the DataFrame:
# (Assumes your CSV loaded the track column as something you can iterate over)
sample_row = df.iloc[0]
track_val = sample_row['track']
# If necessary, parse the track value if it's a string.
track_data = parse_track(track_val)
if track_data is not None:
    metrics = compute_track_metrics(track_data)
    print("Aggregated metrics for first flight track:")
    print(metrics)


Aggregated metrics for first flight track:
{'time_min': np.float64(1735711200.0), 'time_max': np.float64(1735732150.0), 'time_avg': np.float64(1735722079.5689654), 'latitude_min': np.float64(25.1404429), 'latitude_max': np.float64(45.4877278), 'latitude_avg': np.float64(35.97304212672414), 'longitude_min': np.float64(12.1860486), 'longitude_max': np.float64(55.4475403), 'longitude_avg': np.float64(33.47274912284483), 'altitude_min': np.float64(-304.8), 'altitude_max': np.float64(11582.4), 'altitude_avg': np.float64(4767.755172413794), 'heading_min': np.float64(2.7702157), 'heading_max': np.float64(357.1997), 'heading_avg': np.float64(239.93036702887932), 'onground_percentage': np.float64(0.0), 'start_time': Timestamp('2025-01-01 06:00:00'), 'end_time': Timestamp('2025-01-01 11:49:10')}


In [13]:

# If you want to compute the metrics for every row in your DataFrame,
# you can add a new column containing these metrics:
def apply_track_metrics(row):
    track_data = parse_track(row['track'])
    if track_data is not None:
        return compute_track_metrics(track_data)
    return None

df['track_metrics'] = df.apply(apply_track_metrics, axis=1)
print(df[['icao24', 'track_metrics']].head())

   icao24                                      track_metrics
0  8963df  {'time_min': 1735711200.0, 'time_max': 1735732...
1  aaa326  {'time_min': 1735711200.0, 'time_max': 1735735...
2  868b50  {'time_min': 1735711200.0, 'time_max': 1735716...
3  78080d  {'time_min': 1735711200.0, 'time_max': 1735719...
4  78052f  {'time_min': 1735711200.0, 'time_max': 1735723...


In [14]:
df.head()

,icao24,callsign,estdepartureairport,estarrivalairport,firstseen,lastseen,flight_duration,track,track_metrics
0,8963df,UAE135,OMDB,LIPZ,1735711201,1735732155,20954,"[array([ 1.73573215e+09, 4.54877278e+01, 1.2...","{'time_min': 1735711200.0, 'time_max': 1735732..."
1,aaa326,AAL124,NaN,KDFW,1735711202,1735735435,24233,"[array([ 1.73573543e+09, 3.28514953e+01, -9.7...","{'time_min': 1735711200.0, 'time_max': 1735735..."
2,868b50,ANA380,NaN,NaN,1735711203,1735716313,5110,"[array([1.73571631e+09, 3.51708674e+01, 1.3507...","{'time_min': 1735711200.0, 'time_max': 1735716..."
3,78080d,CQH6194,VTBD,NaN,1735711204,1735719171,7967,"[array([1.73571917e+09, 2.29406635e+01, 1.1444...","{'time_min': 1735711200.0, 'time_max': 1735719..."
4,78052f,CXA833,ZSAM,VTBS,1735711204,1735723919,12715,"[array([1.73572392e+09, 1.36619110e+01, 1.0072...","{'time_min': 1735711200.0, 'time_max': 1735723..."


In [15]:
# Expand the track_metrics column (which contains dictionaries) into separate columns
track_metrics_df = pd.json_normalize(df['track_metrics'])

# Concatenate the new columns with the original DataFrame
df = pd.concat([df, track_metrics_df], axis=1)

# Optionally, drop the original track_metrics column if it's no longer needed
df.drop(columns=['track_metrics'], inplace=True)

# Display the DataFrame with the new columns
print(df.head())

   icao24  callsign estdepartureairport estarrivalairport   firstseen  \
0  8963df  UAE135                  OMDB              LIPZ  1735711201   
1  aaa326  AAL124                   NaN              KDFW  1735711202   
2  868b50  ANA380                   NaN               NaN  1735711203   
3  78080d  CQH6194                 VTBD               NaN  1735711204   
4  78052f  CXA833                  ZSAM              VTBS  1735711204   

     lastseen  flight_duration  \
0  1735732155            20954   
1  1735735435            24233   
2  1735716313             5110   
3  1735719171             7967   
4  1735723919            12715   

                                               track      time_min  \
0  [array([ 1.73573215e+09,  4.54877278e+01,  1.2...  1.735711e+09   
1  [array([ 1.73573543e+09,  3.28514953e+01, -9.7...  1.735711e+09   
2  [array([1.73571631e+09, 3.51708674e+01, 1.3507...  1.735711e+09   
3  [array([1.73571917e+09, 2.29406635e+01, 1.1444...  1.735711e+09   
4  [ar

In [16]:
df.head()

,icao24,callsign,estdepartureairport,estarrivalairport,firstseen,lastseen,flight_duration,track,time_min,time_max,...,longitude_avg,altitude_min,altitude_max,altitude_avg,heading_min,heading_max,heading_avg,onground_percentage,start_time,end_time
0,8963df,UAE135,OMDB,LIPZ,1735711201,1735732155,20954,"[array([ 1.73573215e+09, 4.54877278e+01, 1.2...",1.735711e+09,1.735732e+09,...,33.472749,-304.8,11582.4,4767.755172,2.770216,357.19970,239.930367,0.000000,2025-01-01 06:00:00,2025-01-01 11:49:10
1,aaa326,AAL124,NaN,KDFW,1735711202,1735735435,24233,"[array([ 1.73573543e+09, 3.28514953e+01, -9.7...",1.735711e+09,1.735735e+09,...,-103.138302,0.0,11887.2,4241.636129,0.000000,359.62552,74.612923,0.000000,2025-01-01 06:00:00,2025-01-01 12:43:50
2,868b50,ANA380,NaN,NaN,1735711203,1735716313,5110,"[array([1.73571631e+09, 3.51708674e+01, 1.3507...",1.735711e+09,1.735716e+09,...,138.807698,2743.2,11582.4,9701.813592,NaN,NaN,NaN,0.000000,2025-01-01 06:00:00,2025-01-01 07:25:10
3,78080d,CQH6194,VTBD,NaN,1735711204,1735719171,7967,"[array([1.73571917e+09, 2.29406635e+01, 1.1444...",1.735711e+09,1.735719e+09,...,101.321667,0.0,11277.6,4630.479070,46.315903,251.63918,131.633184,0.000000,2025-01-01 06:00:00,2025-01-01 08:12:50
4,78052f,CXA833,ZSAM,VTBS,1735711204,1735723919,12715,"[array([1.73572392e+09, 1.36619110e+01, 1.0072...",1.735711e+09,1.735724e+09,...,104.518985,-304.8,10363.2,2706.413793,NaN,NaN,NaN,4.137931,2025-01-01 06:00:00,2025-01-01 09:32:00


In [17]:
# drop track column
df.drop(columns=['track'], inplace=True)
df.head()


,icao24,callsign,estdepartureairport,estarrivalairport,firstseen,lastseen,flight_duration,time_min,time_max,time_avg,...,longitude_avg,altitude_min,altitude_max,altitude_avg,heading_min,heading_max,heading_avg,onground_percentage,start_time,end_time
0,8963df,UAE135,OMDB,LIPZ,1735711201,1735732155,20954,1.735711e+09,1.735732e+09,1.735722e+09,...,33.472749,-304.8,11582.4,4767.755172,2.770216,357.19970,239.930367,0.000000,2025-01-01 06:00:00,2025-01-01 11:49:10
1,aaa326,AAL124,NaN,KDFW,1735711202,1735735435,24233,1.735711e+09,1.735735e+09,1.735733e+09,...,-103.138302,0.0,11887.2,4241.636129,0.000000,359.62552,74.612923,0.000000,2025-01-01 06:00:00,2025-01-01 12:43:50
2,868b50,ANA380,NaN,NaN,1735711203,1735716313,5110,1.735711e+09,1.735716e+09,1.735713e+09,...,138.807698,2743.2,11582.4,9701.813592,NaN,NaN,NaN,0.000000,2025-01-01 06:00:00,2025-01-01 07:25:10
3,78080d,CQH6194,VTBD,NaN,1735711204,1735719171,7967,1.735711e+09,1.735719e+09,1.735712e+09,...,101.321667,0.0,11277.6,4630.479070,46.315903,251.63918,131.633184,0.000000,2025-01-01 06:00:00,2025-01-01 08:12:50
4,78052f,CXA833,ZSAM,VTBS,1735711204,1735723919,12715,1.735711e+09,1.735724e+09,1.735721e+09,...,104.518985,-304.8,10363.2,2706.413793,NaN,NaN,NaN,4.137931,2025-01-01 06:00:00,2025-01-01 09:32:00


In [ ]:

# save to csv
df.to_csv('../../data/flight_data/flight_data_with_metrics.csv', index=False)

# save a sample of the dataframe
df.sample(1000).to_csv('../../data/flight_data/flight_data_sample.csv', index=False)
